In [5]:
pip install pycryptodome


   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 10.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import socket
from Crypto.Cipher import DES
from Crypto.Util.Padding import unpad  # Used for proper decryption padding handling

def des_decrypt(ciphertext, key):
    des_cipher = DES.new(key, DES.MODE_ECB)
    return unpad(des_cipher.decrypt(ciphertext), DES.block_size)  # Ensures proper padding removal

def server_program():
    host = "127.0.0.2"  # Server IP address
    port = 3010 # Port number
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(5)
    
    print("Server listening on", host, ":", port)
    
    conn, address = server_socket.accept()
    print("Connection from:", str(address))
    
    # Diffie-Hellman key exchange
    p = 23
    g = 5
    server_private_key = 6
    server_public_key = (g ** server_private_key) % p
    conn.send(str(server_public_key).encode())
    
    client_public_key = int(conn.recv(1024).decode())
    shared_secret_key = (client_public_key ** server_private_key) % p
    
    key = str(shared_secret_key).zfill(8).encode()  # Convert key to 8-byte string
    print("Shared secret key:", key.decode())  # Decode for human-readable output
    
    while True:
        data = conn.recv(1024)
        if not data:
            break
        print("Received encrypted message:", data)
        
        try:
            decrypted_data = des_decrypt(data, key)
            print("Decrypted message:", decrypted_data.decode())
        except Exception as e:
            print("Decryption error:", e)  # Handles decryption errors gracefully
    
    conn.close()

if __name__ == '__main__':
    server_program()


Server listening on 127.0.0.2 : 3010
Connection from: ('127.0.0.1', 51073)
Shared secret key: 00000002
Received encrypted message: b'd\xa6^\xb0\xb9\xf4p\xd2\xcb\xa1xpq-\xbd\x85'
Decrypted message: b'00000002'
